<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
    
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
Author: [Yury Kashnitskiy](https://yorko.github.io) (@yorko). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

## <center> Assignment 4. Sarcasm detection with logistic regression
    
We'll be using the dataset from the [paper](https://arxiv.org/abs/1704.05579) "A Large Self-Annotated Corpus for Sarcasm" with >1mln comments from Reddit, labeled as either sarcastic or not. A processed version can be found on Kaggle in a form of a [Kaggle Dataset](https://www.kaggle.com/danofer/sarcasm).

Sarcasm detection is easy. 
<img src="https://habrastorage.org/webt/1f/0d/ta/1f0dtavsd14ncf17gbsy1cvoga4.jpeg" />

In [7]:
# some necessary imports
import os
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from matplotlib import pyplot as plt
%load_ext blackcellmagic
from loguru import logger
import snoop

In [12]:
train_df = pd.read_csv('train-balanced-sarcasm.csv.zip')

In [13]:
train_df.head()

,label,comment,author,subreddit,score,ups,downs,date,created_utc,parent_comment
0,0,NC and NH.,Trumpbart,politics,2,-1,-1,2016-10,2016-10-16 23:55:23,"Yeah, I get that argument. At this point, I'd ..."
1,0,You do know west teams play against west teams...,Shbshb906,nba,-4,-1,-1,2016-11,2016-11-01 00:24:10,The blazers and Mavericks (The wests 5 and 6 s...
2,0,"They were underdogs earlier today, but since G...",Creepeth,nfl,3,3,0,2016-09,2016-09-22 21:45:37,They're favored to win.
3,0,"This meme isn't funny none of the ""new york ni...",icebrotha,BlackPeopleTwitter,-8,-1,-1,2016-10,2016-10-18 21:03:47,deadass don't kill my buzz
4,0,I could use one of those tools.,cush2push,MaddenUltimateTeam,6,-1,-1,2016-12,2016-12-30 17:00:13,Yep can confirm I saw the tool they use for th...


In [14]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype 
---  ------          --------------    ----- 
 0   label           1010826 non-null  int64 
 1   comment         1010773 non-null  object
 2   author          1010826 non-null  object
 3   subreddit       1010826 non-null  object
 4   score           1010826 non-null  int64 
 5   ups             1010826 non-null  int64 
 6   downs           1010826 non-null  int64 
 7   date            1010826 non-null  object
 8   created_utc     1010826 non-null  object
 9   parent_comment  1010826 non-null  object
dtypes: int64(4), object(6)
memory usage: 77.1+ MB


Some comments are missing, so we drop the corresponding rows.

In [15]:
train_df.dropna(subset=['comment'], inplace=True)
train_df['label'].value_counts()
train_texts, valid_texts, y_train, y_valid = \
        train_test_split(train_df['comment'], train_df['label'], random_state=17)

In [66]:
valid_texts

469600    Starting to feel pretty fucking tired of all t...
639137    It's like that label actually has no meaning b...
240293    Mained Fiora - Reworked Mained AP Tristana - W...
702254    Yeah lol that's right they wouldn't let black ...
889040              No, he made the thread asking jokingly.
                                ...                        
23988     Well the devs kind of stopped working on the o...
408769           totally balanced, not game breaking at all
688966    No amount of money can take that experience away.
107936    But guys, you only think D2 was good because o...
767875    BUT HE'S A CAREER THIRD LINER AND WE SHOULD DU...
Name: comment, Length: 252694, dtype: object

In [4]:
from collections import defaultdict
from wordcloud import WordCloud, STOPWORDS

In [40]:
import string

In [51]:
def remove_punct(tokens):
    cleared = []
    for token in tokens:
        token = token.translate(str.maketrans("", "", string.punctuation))
        cleared.append(token)
    return cleared

In [52]:
def generate_ngrams(text, n_gram=1):
    token = [token for token in text.lower().split(" ") if token != "" if token not in STOPWORDS]
    token = remove_punct(token)
    ngrams = zip(*[token[i:] for i in range(n_gram)])
    return [" ".join(ngram) for ngram in ngrams]

In [53]:
def sarcasm_word_count():
    wc = defaultdict(int)
    sarcasms = train_df[train_df["label"] == 1]
    for comment in sarcasms["comment"]:
        for word in generate_ngrams(comment):
            wc[word] += 1
    return wc

In [54]:
sc = sarcasm_word_count()
sc = pd.DataFrame(sorted(sc.items(), key=lambda x: x[1])[::-1])
sc.columns = ['word', 'wordcount']

In [60]:
def not_sarcasm_word_count():
    wc = defaultdict(int)
    not_sarcasms = train_df[train_df["label"] == 0]
    for comment in not_sarcasms["comment"]:
        for word in generate_ngrams(comment):
            wc[word] += 1
    return wc

In [61]:
nsc = not_sarcasm_word_count()
nsc = pd.DataFrame(sorted(nsc.items(), key=lambda x: x[1])[::-1])
nsc.columns = ['word', 'wordcount']

In [62]:
nsc

,word,wordcount
0,,26402
1,one,17918
2,think,15384
3,people,15245
4,will,12546
...,...,...
147380,yahudi,1
147381,rfuckyouididthemath,1
147382,kiiiiiiiil,1
147383,preflashpoint,1


In [63]:
from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

In [67]:
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tfidf_vec.fit_transform(train_texts.values.tolist() + valid_texts.values.tolist())
train_tfidf = tfidf_vec.transform(train_texts.values.tolist())
test_tfidf = tfidf_vec.transform(valid_texts.values.tolist())

In [ ]:
def runModel(train_X, train_y, test_X, test_y, test_X2):
    model = linear_model.LogisticRegression(C=5., solver='sag')
    model.fit(train_X, train_y)
    pred_test_y = model.predict_proba(test_X)[:,1]
    pred_test_y2 = model.predict_proba(test_X2)[:,1]
    return pred_test_y, pred_test_y2, model

## Tasks:
1. Analyze the dataset, make some plots. This [Kernel](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-qiqc) might serve as an example
2. Build a Tf-Idf + logistic regression pipeline to predict sarcasm (`label`) based on the text of a comment on Reddit (`comment`).
3. Plot the words/bigrams which a most predictive of sarcasm (you can use [eli5](https://github.com/TeamHG-Memex/eli5) for that)
4. (optionally) add subreddits as new features to improve model performance. Apply here the Bag of Words approach, i.e. treat each subreddit as a new feature.

## Links:
  - Machine learning library [Scikit-learn](https://scikit-learn.org/stable/index.html) (a.k.a. sklearn)
  - Kernels on [logistic regression](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-2-classification) and its applications to [text classification](https://www.kaggle.com/kashnitsky/topic-4-linear-models-part-4-more-of-logit), also a [Kernel](https://www.kaggle.com/kashnitsky/topic-6-feature-engineering-and-feature-selection) on feature engineering and feature selection
  - [Kaggle Kernel](https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle) "Approaching (Almost) Any NLP Problem on Kaggle"
  - [ELI5](https://github.com/TeamHG-Memex/eli5) to explain model predictions